# Simple SWMM Model Calibration Example

This example demonstrates how to set up and run a basic calibration using optswmm.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent.parent))
# Import optswmm modules

from optswmm.utils.calparams import CalParam, CalParams
from optswmm.utils.calibutils import calibrate
from optswmm.utils.optconfig import OptConfig
from optswmm.utils.runutils import OptRun
from swmmio import Model

## 1. Set up file paths

In [2]:
# Define paths relative to the optswmm root directory
optswmm_root = Path.cwd().parent.parent  # Assuming we're in examples folder
test_dir = optswmm_root / "tests" / "fixtures"

# Model file (you'll need to ensure this exists in your test fixtures)
model_file = test_dir / "sample_models" / "Example1.inp"

# Data files
forcing_data = test_dir / "sample_data" / "forcing_data.csv"
target_data = test_dir / "sample_data" / "observed_data.csv"

# Configuration file
config_file = test_dir / "configs" / "sample_optconfig.yaml"

print(f"Model file: {model_file}")
print(f"Config file: {config_file}")
print(f"Test directory: {test_dir}")

Model file: c:\Users\everett\Documents\GitHub\optswmm\tests\fixtures\sample_models\Example1.inp
Config file: c:\Users\everett\Documents\GitHub\optswmm\tests\fixtures\configs\sample_optconfig.yaml
Test directory: c:\Users\everett\Documents\GitHub\optswmm\tests\fixtures


In [3]:

from pyswmm import Simulation, Output
from swmm.toolkit import shared_enum

from swmmio import Model
model = Model(str(model_file))
outfall_node = model.inp.outfalls.index[0]

with Simulation(str(model_file)) as sim:
    sim.execute()

with Output(str(model_file.with_suffix(".out"))) as out:
    res = out.node_series(outfall_node, shared_enum.NodeAttribute.TOTAL_INFLOW)
out.close()
df = pd.Series(res).sort_index().rename('flow').to_frame()
df.index = pd.to_datetime(df.index)
df.index.name = 'datetime'


... SWMM Version 5.2.4
... Run Complete

In [ ]:
# Create calibration parameters
cal_params = CalParams([
    # Subcatchment parameters
    CalParam(
        section='subcatchments',
        attribute='PercImperv',
        lower_rel=-0.2,  # 20% decrease
        upper_rel=0.3,   # 30% increase
        distributed=True,
    ),
    CalParam(
        section='subcatchments',
        attribute='Width',
        lower_rel=-0.15,
        upper_rel=0.25,
        distributed=True,
    ),
    # Conduit roughness
    CalParam(
        section='conduits',
        attribute='Roughness',
        lower_rel=-0.3,
        upper_rel=0.5,
        distributed=True,
    )
])

model_vals = [10, 5, 1]
for ii in range(len(cal_params)):
    cal_params[ii].model_val = model_vals[ii]
spc = CalParams.make_simulation_preconfig(model=model)
    

print(f"Created {len(cal_params)} calibration parameters:")
for i, cp in enumerate(cal_params):
    print(f"  {i+1}. {cp.section}.{cp.attribute} (mode: {cp.mode}, distributed: {cp.distributed})")

## 2. Create sample data (if not exists)

In [ ]:
# Create sample observed flow data
if not target_data.exists():
    target_data.parent.mkdir(parents=True, exist_ok=True)
    
    # Generate synthetic observed flow data
    dates = pd.date_range('1998-01-01 00:00', periods=24, freq='H')
    
    # Create multi-level columns for nodes and variables
    nodes = ['J1', 'J2', 'J3']
    variables = ['flow']
    
    columns = pd.MultiIndex.from_product([variables, nodes], names=['variable', 'node'])
    
    # Generate synthetic flow data with some correlation to rainfall
    forcing_df = pd.read_csv(forcing_data, index_col=0, parse_dates=True)
    base_flow = 0.5  # m3/s base flow
    
    target_df = pd.DataFrame(index=dates, columns=columns)
    
    for node in nodes:
        # Simple rainfall-runoff relationship
        runoff = forcing_df['rainfall'] * 0.1 * np.random.uniform(0.8, 1.2)
        flow = base_flow + runoff + np.random.normal(0, 0.1, size=48)
        flow[flow < 0] = base_flow  # Ensure non-negative flows
        target_df[('flow', node)] = flow
    
    target_df.to_csv(target_data)
    print(f"Created sample target data: {target_data}")

NameError: name 'target_data' is not defined

## 3. Define calibration parameters

Created 3 calibration parameters:
  1. subcatchments.PercImperv (mode: multiplicative, distributed: True)
  2. subcatchments.Width (mode: multiplicative, distributed: True)
  3. conduits.Roughness (mode: multiplicative, distributed: True)


## 4. Create optimization configuration

In [ ]:
# Create optimization configuration
config_data = {
    'run_folder': str(Path("runs").resolve()),
    'model_file': str(model_file.resolve()),
    #'forcing_data_file': str(forcing_data.resolve()),
    'target_data_file': str(target_data.resolve()),
    'algorithm': 'differential-evolution',
    'algorithm_options': {
        'maxiter': 50,
        'popsize': 10,
        'seed': 42
    },
    'score_function': ['nse'],  # Nash-Sutcliffe Efficiency
    'target_variables': ['flow'],
    'hierarchical': False,
    'normalize': False,
    'warmup_length': 0,
}

# Save configuration to file
config_file.parent.mkdir(parents=True, exist_ok=True)
import yaml

with open(config_file, 'w') as f:
    yaml.dump(config_data, f, default_flow_style=False)

print(f"Created optimization configuration: {config_file}")
print(f"Algorithm: {config_data['algorithm']}")
print(f"Max iterations: {config_data['algorithm_options']['maxiter']}")
print(f"Score function: {config_data['score_function']}")

NameError: name 'Path' is not defined

## 5. Load and preprocess calibration parameters

In [ ]:
# Load optimization configuration
opt_config = OptConfig(config_file=config_file)

# Preprocess calibration parameters
cal_params_processed = cal_params.preprocess(opt_config)

print(f"Processed {len(cal_params_processed)} calibration parameters")
print(f"Total active parameters: {sum(1 for cp in cal_params_processed if cp.active)}")

# Display parameter bounds
bounds = cal_params_processed.get_bounds()
flattened_bounds = cal_params_processed.flatten_bounds(bounds)
print(f"\nParameter bounds (flattened): {len(flattened_bounds)} parameters")
for i, bound in enumerate(flattened_bounds[:5]):  # Show first 5
    print(f"  Parameter {i+1}: {bound}")
if len(flattened_bounds) > 5:
    print(f"  ... and {len(flattened_bounds) - 5} more")

## 6. Run the calibration

In [ ]:
# Run calibration
print("Starting calibration...")
print("This may take several minutes depending on model complexity and iteration count.")

try:
    calibrate(opt_config=opt_config, cal_params=cal_params_processed)
    print("\n✅ Calibration completed successfully!")
    
except Exception as e:
    print(f"\n❌ Calibration failed with error: {e}")
    print("This might be due to missing model file or data incompatibility.")
    print("Please check that the model file exists and is compatible with the test data.")

Starting calibration...
This may take several minutes depending on model complexity and iteration count.


c:\Users\everett\Documents\GitHub\optswmm\optswmm\utils\standardization.py:215: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True)
c:\Users\everett\Documents\GitHub\optswmm\optswmm\utils\standardization.py:169: UserWarning: Target data 'Index(['timestamp', 'value'], dtype='object')' has less than 1000 timesteps
  warnings.warn(f"Target data '{tgt.columns}' has less than 1000 timesteps", UserWarning)
c:\Users\everett\Documents\GitHub\optswmm\optswmm\utils\optconfig.py:111: UserWarning: Uncalibrated model file C:\Users\everett\Documents\GitHub\optswmm\tests\fixtures\sample_models\Example1_uncal.inp already exists. It will be overwritten.
  warnings.warn(f"Uncalibrated model file {uncalibrated_model_file} already exists. It will be overwritten.")
c:\Users\everett\Documents\GitHub\optswmm\optswmm\utils\s


❌ Calibration failed with error: 
  ERROR 191: simulation start date comes after ending date.
This might be due to missing model file or data incompatibility.
Please check that the model file exists and is compatible with the test data.


## 7. Analyze results

In [ ]:
# Load and analyze optimization results
try:
    opt_run = OptRun(config_file=config_file)
    
    print(f"Results directory: {opt_run.dir}")
    print(f"Config loaded from: {opt_run.config_file}")
    
    # Check if results files exist
    results_files = {
        'Parameters': opt_run.dir / 'results_params.txt',
        'Scores': opt_run.dir / 'results_scores.txt',
        'Calibration Parameters': opt_run.dir / 'calibration_parameters.csv',
        'Calibrated Model': opt_run.dir / 'calibrated_model.inp'
    }
    
    print("\nResults files:")
    for name, file_path in results_files.items():
        exists = "✅" if file_path.exists() else "❌"
        print(f"  {exists} {name}: {file_path.name}")
    
    # Load and display optimization scores if available
    scores_file = results_files['Scores']
    if scores_file.exists():
        scores_df = pd.read_csv(scores_file)
        print(f"\nOptimization progress:")
        print(f"  Total iterations: {len(scores_df)}")
        print(f"  Best score: {scores_df['score'].min():.4f}")
        print(f"  Final score: {scores_df['score'].iloc[-1]:.4f}")
        
        # Show score improvement
        initial_score = scores_df['score'].iloc[0]
        final_score = scores_df['score'].min()
        improvement = ((initial_score - final_score) / abs(initial_score)) * 100
        print(f"  Improvement: {improvement:.1f}%")
        
except Exception as e:
    print(f"Could not load optimization results: {e}")
    print("This is normal if the calibration failed or is still running.")

Could not load optimization results: OptRun.__init__() got an unexpected keyword argument 'config_file'
This is normal if the calibration failed or is still running.


## 8. Plot results (if available)

In [ ]:
# Plot optimization progress
try:
    import matplotlib.pyplot as plt
    
    # Load results
    scores_file = opt_run.dir / 'results_scores.txt'
    params_file = opt_run.dir / 'results_params.txt'
    
    if scores_file.exists() and params_file.exists():
        scores_df = pd.read_csv(scores_file)
        params_df = pd.read_csv(params_file)
        
        # Create subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        
        # Plot score evolution
        ax1.plot(scores_df['iter'], scores_df['score'], 'b-', linewidth=2)
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Objective Function Value')
        ax1.set_title('Optimization Progress')
        ax1.grid(True, alpha=0.3)
        
        # Plot parameter evolution (first few parameters)
        param_cols = [col for col in params_df.columns if col not in ['iter', 'score']]
        for i, col in enumerate(param_cols[:5]):  # Plot first 5 parameters
            ax2.plot(params_df['iter'], params_df[col], label=f'Param {i+1}', alpha=0.7)
        
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Parameter Value')
        ax2.set_title('Parameter Evolution')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("📊 Optimization plots generated successfully!")
    else:
        print("Results files not found - cannot generate plots")
        
except ImportError:
    print("Matplotlib not available - skipping plots")
except Exception as e:
    print(f"Error generating plots: {e}")

Error generating plots: name 'opt_run' is not defined


## 9. Summary

In [ ]:
print("\n" + "="*50)
print("CALIBRATION EXAMPLE SUMMARY")
print("="*50)
print(f"Model file: {model_file.name}")
print(f"Configuration: {config_file.name}")
print(f"Calibration parameters: {len(cal_params)}")
print(f"Algorithm: {config_data['algorithm']}")
print(f"Score function: {config_data['score_function'][0]}")

try:
    if opt_run.dir.exists():
        print(f"\nResults saved to: {opt_run.dir}")
        
        # List key output files
        key_files = [
            'calibrated_model.inp',
            'results_scores.txt', 
            'results_params.txt',
            'calibration_parameters.csv'
        ]
        
        print("\nKey output files:")
        for filename in key_files:
            filepath = opt_run.dir / filename
            status = "✅" if filepath.exists() else "❌"
            print(f"  {status} {filename}")
            
except:
    print("\nNo results directory found - calibration may have failed")

print("\n" + "="*50)
print("Example completed!")
print("\nNext steps:")
print("- Examine the calibrated model file")
print("- Analyze parameter sensitivity")
print("- Validate results with independent data")
print("- Adjust calibration settings if needed")


CALIBRATION EXAMPLE SUMMARY
Model file: Example1.inp
Configuration: sample_optconfig.yaml
Calibration parameters: 3
Algorithm: differential-evolution
Score function: nse

No results directory found - calibration may have failed

Example completed!

Next steps:
- Examine the calibrated model file
- Analyze parameter sensitivity
- Validate results with independent data
- Adjust calibration settings if needed
